In [15]:
from AniList_extractor import AniListUser
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [2]:
df_completed, df_plan_to_watch = AniListUser('thems22').get_dfs()

In [29]:
df_plan_to_watch['english'] = np.where(df_plan_to_watch['english'].isna(), df_plan_to_watch['romaji'], df_plan_to_watch['english'])
df_plan_to_watch = df_plan_to_watch.dropna()

df_completed['english'] = np.where(df_completed['english'].isna(), df_completed['romaji'], df_completed['english'])

In [30]:
df_completed['watched'] = np.int8(1)
df_plan_to_watch['watched'] = np.int8(0)

/tmp/ipykernel_964/3464976671.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_plan_to_watch['watched'] = np.int8(0)


In [94]:
df_watched = pd.concat([df_completed[['description', 'watched', 'english']], df_plan_to_watch[['description', 'watched', 'english']]])
df_watched = df_watched.reset_index()

In [95]:
df_watched['description'] = df_watched['description'].str.replace('<.*?>', '', regex=True).str.replace('\n', '')

In [96]:
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df=0, stop_words='english')

In [97]:
tfidf_matrix = tf.fit_transform(df_watched['description'])

In [98]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [99]:
max_idx = df_watched[df_watched['watched']==1].index[-1]

In [100]:
plan_to_watch_cosine = cosine_sim[max_idx::, max_idx::].shape

In [101]:
df_plan_to_watch.shape

(301, 242)

In [102]:
titles = df_watched['english']
indices = pd.Series(df_watched.index, index=df_watched['english'])

In [119]:
def get_recommendations(title, df):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores
    movie_indices = [i[0] for i in sim_scores]
    df['rank'] = pd.DataFrame(list(enumerate(movie_indices))).set_index(1)
    df = df[df['watched']==0]
    return df.sort_values(by='rank')['english'][0:30]

In [137]:
df_watched[df_watched['english'].str.contains('naruto', case=False)]

,index,description,watched,english,rank
162,162,"Naruto Uzumaki, a hyperactive and knuckle-head...",1,Naruto,262
163,163,Naruto: Shippuuden is the continuation of the ...,1,Naruto: Shippuden,71


In [143]:
df_completed[df_completed['score']==7]['english']

6                                           ALDNOAH.ZERO
9                                                Another
11                                         Blue Exorcist
12                           The Heroic Legend of Arslan
18                                          Black Lagoon
30                                               Clannad
31                                  Clannad: After Story
44                                            Death Note
59     Fate/stay night [Heaven's Feel] I. presage flower
60     Fate/stay night [Heaven's Feel] II. lost butte...
61      Fate/stay night [Heaven’s Feel] III. spring song
64                                             Fate/Zero
68     Fullmetal Alchemist: The Movie - Conqueror of ...
74                        GOBLIN SLAYER -GOBLIN’S CROWN-
81                                        Haibane Renmei
86                                  High School DxD BorN
92                                Hunter x Hunter (2011)
93                             

In [147]:
get_recommendations("Texhnolyze", df_watched)

523                                           Texhnolyze
482                                      Night Raid 1931
450                         Nura: Rise of the Yokai Clan
330                               Fate/EXTRA Last Encore
545                                          Wolf's Rain
299                                          City Hunter
462                Planetarian: Storyteller of the Stars
524                     A Certain Scientific Accelerator
328                                           Fairy gone
443    Natsume's Book of Friends the Movie: Ephemeral...
279                  Baka and Test - Summon the Beasts 2
327                                           Ergo Proxy
368                                          HINAMATSURI
452                                        Wolf Children
320                                             DRIFTERS
403                                Wise Man’s Grandchild
310                                DARLING in the FRANXX
536           Star Blazers: Spa